In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from src import data_cleaning as dc

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LSTM, Dropout

from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
class lstm(object):
    def __init__(self, playerID):
        self.playerID = playerID
        self.X, self.y = format_data()
        self.X = dict_conver
        
    def format_data(self):
        df = pd.read_csv('src/data/uncondensed.csv')
        df.drop(['Unnamed: 0', 'yearID'], axis=1, inplace=True)
        df = pd.get_dummies(df, columns=['pos'])
        df.set_index('playerID', inplace=True, drop=True)
        X = df.groupby('playerID').head(6)
        y = pd.DataFrame(df['avg'].groupby('playerID').tail(6))
        
        return X, y
        
    def dict_convert(self, df):
        length = pd.value_counts(df.index)[0]
        def_dict = defaultdict(dict)
        if length == 1:
            return df.T.to_dict()
        for i in range(length):
            year = 'year' + str(i + 1)
            def_dict[year] = df.groupby('playerID').nth(i).T.to_dict('index')
        return def_dict

    def normalize_feats(self, df):
        cols = [col for col in X.columns if 'pos' not in col]
        cols.remove('avg')
        df[cols] = normalize(df[cols])
        return df

    def fit_lstm(self, X_train, y_train):
        lstm_model = Sequential()
        lstm_model.add(LSTM(28, input_shape=(6, 22), return_sequences=True))
        lstm_model.add(BatchNormalization())
        lstm_model.add(Dropout(.3))
        lstm_model.add(LSTM(28))
        lstm_model.add(BatchNormalization())
        lstm_model.add(Dense(64))
        lstm_model.add(Dense(1))
        lstm_model.compile(loss='mean_squared_error', optimizer='adam')
        lstm_model.fit(X_train, y_train, verbose=1, epochs=20, batch_size=10)

        return lstm_model

    def display_results(self, model, X_test, y_test):
        y_pred = model.predict(X_test)
        df = pd.DataFrame(y_test)
        df.columns = ['actual']
        df['predicted'] = y_pred
        df = df.round(3)
        df['difference'] = (df['actual'] - df['predicted']).abs()
        return df